In [ ]:
# 필요 라이브러리 설치
!pip install "peft"
!pip install "transformers==4.30" "datasets==2.9.0" "accelerate" "evaluate==0.4.0" "bitsandbytes" loralib --upgrade --quiet
!pip install rouge-score tensorboard py7zr

# Load dataset

- samsum dataset : 16k messenger-like conversations with summaries.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("samsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-xxl"

tokenizer = AutoTokenizer.from_pretrained(model_id)

학습하기 전, 적절한 max_length 설정을 위한 데이터 탐색



In [ ]:
from datasets import concatenate_datasets
import numpy as np

tokenized_inputs = concatenate_datasets([dataset['train'], dataset['test']]).map(
                                                                                    lambda x: tokenizer(x['dialogue'], truncation=True),
                                                                                    batched=True,
                                                                                    remove_columns=['dialogue', 'summary']
                                                                                )
input_lengths = [len(x) for x in tokenized_inputs['input_ids']]
# 대화문의 최대 길이의 85분위 사용
max_source_length = int(np.percentile(input_lengths, 85))
print(f"Max source length: {max_source_length}")


tokenized_inputs = concatenate_datasets([dataset['train'], dataset['test']]).map(
                                                                                    lambda x: tokenizer(x['summary'], truncation=True),
                                                                                    batched=True,
                                                                                    remove_columns=['dialogue', 'summary']
                                                                                )
target_lengths = [len(x) for x in tokenized_inputs['input_ids']]
# 요약문의 최대 길이의 90분위 사용
max_target_length = int(np.percentile(target_lengths, 90))
print(f"Max target length: {max_target_length}")

prerocess function 정의

In [ ]:
def preprocess_function(sample, padding="max_length"):
    # T5 모델을 위한 input 변경
    print(sample)
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # Tokenize input
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize target(text_target 사용)
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)


    # padding 토큰이 loss 값을 계산할 때 무시되기 위해서는 pad_token_id가 아닌 -100으로 바꿔주어야 함
    if padding == "max_length":
        labels['input_ids'] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels['input_ids']
        ]

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function,
                                batched=True,
                                remove_columns=["dialogue", "summary", "id"])


print(f"Columns of tokenized dataset {list(tokenized_dataset['train'].features)}")

# 쉽게 불러올 수 있도록 디스크에 저장
tokenized_dataset['train'].save_to_disk('./data/train')
tokenized_dataset['test'].save_to_disk('./data/eval')

# LoRA를 이용한 T5의 Fine-tuning

[bitsanbytes.LLM.int8()](https://huggingface.co/blog/hf-bitsandbytes-integration)을 이용해 LLM을 int8로 quantize할 수 있으며 이 방식을 통해 메모리를 크게 줄일 수 있다.

> Google Colab에서 *load_in_8bit*인자의 GPU 용량 문제로 인해 *load_in_4bit*로 대체


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_id = "philschmid/flan-t5-xxl-sharded-fp16"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")

### k-bit 학습 준비

- 결과적으로 전체 모델의 약 0.16%의 파라미터만을 학습하게 된다.

- 따라서, 메모리 문제 없이 모델을 fine-tuning 할 수 있게 된다.

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Lora Config 정의
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# 모델 학습 준비
model = prepare_model_for_kbit_training(model)

# LoRA adaptor 추가
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

### DataCollator

- 동적 padding을 위한 객체

In [ ]:
from transformers import DataCollatorForSeq2Seq

label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 # 해당 인자의 배수로 padding이 채워짐.(대체적으로 8의 배수가 좋다고 함)
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir = 'lora-flan-t5-xxl'

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Trainer 생성
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
)

# True인 경우 과거 키 값을 사용, 훈련 과정에서는 필요하지 않음
model.config.use_cache = False

T5의 경우 몇몇 layer들은 안정성을 위해 *float32*로 유지됨

In [ ]:
trainer.train()

In [ ]:
# 모델 저장
trainer.model.save_pretrained('results')
tokenizer.save_pretrained('results')

# LoRA FLAN-T5를 사용한 평가 및 추론

- 저장된 모델의 크기는 약 70M에 불과함

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# PEFT config 불러오기
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# Base Model, Tokenizer 불러오기
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  load_in_4bit=True,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# LoRA 모델 불러오기
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

## sample을 이용한 추론 테스트

In [ ]:
from datasets import load_dataset
from random import randrange


# 데이터 불러오기 및 샘플링
dataset = load_dataset("samsum")
sample = dataset['test'][randrange(len(dataset["test"]))]
input_ids = tokenizer(sample["dialogue"], return_tensors="pt", truncation=True).input_ids.cuda()

# 모델 추론 결과
outputs = model.generate(input_ids=input_ids, max_new_tokens=10, do_sample=True, top_p=0.9)


print(f"input sentence: {sample['dialogue']}\n{'---'* 20}")
print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

## 모델 평가

In [ ]:
# 시간이 많이 소요되어 10개의 데이터만 사용
from datasets import Dataset

test_dataset = Dataset.from_dict(test_dataset[:10])

In [ ]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

metric = evaluate.load('rouge')

def evaluate_peft_model(sample, max_target_length=50):
  # 요약 결과
  outputs = model.generate(input_ids=sample['input_ids'].unsqueeze(0).cuda(),
                           do_sample=True,
                           top_p=0.9,
                           max_new_tokens=max_target_length,
                           )

  prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)

  # -100 토큰 pad 토큰으로 변경 및 디코딩
  labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
  labels = tokenizer.decode(labels, skip_special_tokens=True)

  return prediction, labels

# 테스트 데이터 불러오기(시간이 많이 소요되어 10개의 데이터만 사용)
test_dataset = load_from_disk('data/eval/').with_format("torch")
test_dataset = Dataset.from_dict(test_dataset[:10]).with_format("torch")


# 추론 시작
predictions, references = [], []
for sample in tqdm(test_dataset):
  p, l = evaluate_peft_model(sample)
  predictions.append(p)
  references.append(l)

# 계산
rouge = metric.compute(predictions=predictions, references=references, use_stemmer=True)

print(f"Rouge1: {rouge['rouge1']* 100:2f}%")
print(f"Rouge2: {rouge['rouge2']* 100:2f}%")
print(f"RougeL: {rouge['rougeL']* 100:2f}%")
print(f"RougeLsum: {rouge['rougeLsum']* 100:2f}%")

Colab으로 학습을 제대로 진행하지 못했음에도 불구하고 rouge1은 약 38%의 평가를 보여줌.